<a href="https://colab.research.google.com/github/malskim1123/CIS-2100-Project-2/blob/main/CIS_2100_project_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install pandas mlxtend

In [23]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder


In [5]:
path = "/content/drive/MyDrive/CIS 2100 proj 2/sales_records.csv"
df = pd.read_csv(path)
baskets_per_store = df.groupby(['StoreID', 'OrderID'])['ProductName'].apply(list)

# **BEST SELLING ITEM IN 10 STORES**

In [38]:
#analysis of the best selling items for each store (chose 10 random stores)
def get_best_selling_item_per_store(df, store_id):
    """
    Returns the best-selling item for a specific store.
    """
    store_data = df[df['StoreID'] == store_id]
    best_selling_item = store_data['ProductName'].value_counts().head(1)
    return best_selling_item

random_stores = random.sample(df['StoreID'].unique().tolist(), 10)

best_selling_items_per_store = []

for store_id in random_stores:
    best_selling_item = get_best_selling_item_per_store(df, store_id)
    best_selling_items_per_store.append((store_id, best_selling_item.index[0], best_selling_item.values[0]))

best_selling_df = pd.DataFrame(best_selling_items_per_store, columns=['StoreID', 'BestSellingItem', 'SalesCount'])

top_10_stores = best_selling_df.sort_values(by='SalesCount', ascending=False).head(10)

for idx, row in top_10_stores.iterrows():
    print(f"Store {row['StoreID']} - Best-selling item: {row['BestSellingItem']} - Sold {row['SalesCount']} times")
    print("-" * 50)

Store 1444 - Best-selling item: Ibanez Guitar - Sold 6 times
--------------------------------------------------
Store 693 - Best-selling item: Canon EOS Camera - Sold 5 times
--------------------------------------------------
Store 944 - Best-selling item: Canon EOS Camera - Sold 5 times
--------------------------------------------------
Store 121 - Best-selling item: Air Jordan 1 - Sold 4 times
--------------------------------------------------
Store 1310 - Best-selling item: Whey Protein Powder - Sold 4 times
--------------------------------------------------
Store 723 - Best-selling item: Canon EOS Camera - Sold 3 times
--------------------------------------------------
Store 1041 - Best-selling item: Air Jordan 1 - Sold 2 times
--------------------------------------------------
Store 2231 - Best-selling item: Burton Snowboard - Sold 2 times
--------------------------------------------------
Store 1710 - Best-selling item: Sony Headphone - Sold 2 times
------------------------------

# **TOP 10 BEST-SELLING ITEMS IN THE CORPORATION**


In [25]:
#best-selling items across the entire organization (top 10 items)
def get_best_selling_items_across_organization(df):
    """
    Returns the top-selling items across the entire organization.
    """
    # Count the frequency of each product across all stores
    best_selling_items = df['ProductName'].value_counts().head(10)  # Top 10 items
    return best_selling_items

# Get the top 10 best-selling items across the entire organization
top_selling_items = get_best_selling_items_across_organization(df)

# Print the results
print("Top 10 Best-Selling Items Across the Corporation:")
print("=" * 50)

for idx, (item, count) in enumerate(top_selling_items.items(), 1):
    print(f"{idx}. {item} - Sold {count} times")
    print("-" * 50)

Top 10 Best-Selling Items Across the Corporation:
1. Ibanez Guitar - Sold 3981 times
--------------------------------------------------
2. Canon EOS Camera - Sold 3930 times
--------------------------------------------------
3. Air Jordan 1 - Sold 3907 times
--------------------------------------------------
4. Camille Rose Conditioner - Sold 3899 times
--------------------------------------------------
5. Burton Snowboard - Sold 3875 times
--------------------------------------------------
6. Carpet - Sold 3857 times
--------------------------------------------------
7. Victus Laptop - Sold 3848 times
--------------------------------------------------
8. Smith Ski Goggles - Sold 3843 times
--------------------------------------------------
9. Sony Headphone - Sold 3805 times
--------------------------------------------------
10. Whey Protein Powder - Sold 3791 times
--------------------------------------------------


# **Top 5 Most Frequent Item Pairs Across 10 Stores**

In [24]:
#Market basket analysis, performed across your corporate data
random_stores = random.sample(baskets_per_store.index.get_level_values(0).unique().tolist(), 10)

report = []

for store_id in random_stores:
    transactions = baskets_per_store[store_id].tolist()

    te = TransactionEncoder()
    te_ary = te.fit(transactions).transform(transactions)

    store_df = pd.DataFrame(te_ary, columns=te.columns_)

    frequent_itemsets = apriori(store_df, min_support=0.05, use_colnames=True)

    frequent_itemsets_sorted = frequent_itemsets.sort_values(by='support', ascending=False)

    top_itemsets = frequent_itemsets_sorted.head(5)

    report.append(f"Top 5 Best-Selling Items for Store {store_id}:\n")
    report.append("="*50)

    for idx, row in top_itemsets.iterrows():
        items = ", ".join(row['itemsets'])
        support = round(row['support'], 4)
        report.append(f"{items} - Purchased together in {support*100}% of orders.")

    report.append("="*50 + "\n")

for line in report:
    print(line)

Top 5 Best-Selling Items for Store 2439:

Nike Sweatpants, Whey Protein Powder - Purchased together in 50.0% of orders.
Nike Sweatpants - Purchased together in 50.0% of orders.
Sony Headphone - Purchased together in 50.0% of orders.
Whey Protein Powder - Purchased together in 50.0% of orders.
Victus Laptop, Smith Ski Goggles, Whey Protein Powder - Purchased together in 25.0% of orders.

Top 5 Best-Selling Items for Store 872:

Air Jordan 1 - Purchased together in 100.0% of orders.
Ibanez Guitar - Purchased together in 100.0% of orders.
Air Jordan 1, Ibanez Guitar - Purchased together in 100.0% of orders.
Air Jordan 1, Ibanez Guitar, Whey Protein Powder - Purchased together in 66.67% of orders.
Air Jordan 1, Whey Protein Powder - Purchased together in 66.67% of orders.

Top 5 Best-Selling Items for Store 1035:

Nike Sweatpants - Purchased together in 100.0% of orders.
Air Jordan 1 - Purchased together in 50.0% of orders.
Sony Headphone, Nike Sweatpants - Purchased together in 50.0% of o

# **TOP 10 STORES WITH HIGHEST BEST SELLERS**

In [32]:
best_selling_items_per_store = df.groupby(['StoreID', 'ProductName']).size().reset_index(name='Count')

best_selling_items_per_store = best_selling_items_per_store.sort_values(by=['StoreID', 'Count'], ascending=[True, False])

top_best_selling_items = best_selling_items_per_store.drop_duplicates('StoreID', keep='first')

top_10_stores = top_best_selling_items.sort_values(by='Count', ascending=False).head(10)

top_10_stores['Rank'] = range(1, len(top_10_stores) + 1)

for idx, row in top_10_stores.iterrows():
    print(f"Top {row['Rank']} - Store {row['StoreID']} - Best-Selling Item: {row['ProductName']} with {row['Count']} purchases")

Top 1 - Store 1678 - Best-Selling Item: Ibanez Guitar with 10 purchases
Top 2 - Store 1167 - Best-Selling Item: Air Jordan 1 with 10 purchases
Top 3 - Store 1537 - Best-Selling Item: Air Jordan 1 with 9 purchases
Top 4 - Store 2137 - Best-Selling Item: Canon EOS Camera with 9 purchases
Top 5 - Store 307 - Best-Selling Item: Ibanez Guitar with 9 purchases
Top 6 - Store 1365 - Best-Selling Item: Burton Snowboard with 9 purchases
Top 7 - Store 1894 - Best-Selling Item: Air Jordan 1 with 9 purchases
Top 8 - Store 2002 - Best-Selling Item: Victus Laptop with 9 purchases
Top 9 - Store 2421 - Best-Selling Item: Smith Ski Goggles with 9 purchases
Top 10 - Store 2177 - Best-Selling Item: Victus Laptop with 9 purchases


# **Top 10 Stores With Highest Total Sales**

In [43]:
df['Price'] = df['Price'].replace({'\$': '', ',': ''}, regex=True).astype(float)

total_sales_per_order = df.groupby(['StoreID', 'OrderID'])['Price'].sum().reset_index()

total_sales_per_store = total_sales_per_order.groupby('StoreID')['Price'].sum().reset_index()

top_10_stores_by_sales = total_sales_per_store.sort_values(by='Price', ascending=False).head(10)

top_10_stores_by_sales['Top'] = range(1, len(top_10_stores_by_sales) + 1)

for idx, row in top_10_stores_by_sales.iterrows():
    print(f"Top {int(row['Top'])} - Store {int(row['StoreID'])} - Total Sales: ${row['Price']:.2f}")
    print("-" * 50)

Top 1 - Store 2177 - Total Sales: $14795.64
--------------------------------------------------
Top 2 - Store 1203 - Total Sales: $12782.69
--------------------------------------------------
Top 3 - Store 1020 - Total Sales: $12740.76
--------------------------------------------------
Top 4 - Store 1365 - Total Sales: $12551.73
--------------------------------------------------
Top 5 - Store 92 - Total Sales: $12527.71
--------------------------------------------------
Top 6 - Store 150 - Total Sales: $12486.74
--------------------------------------------------
Top 7 - Store 923 - Total Sales: $12401.64
--------------------------------------------------
Top 8 - Store 1537 - Total Sales: $12333.72
--------------------------------------------------
Top 9 - Store 1500 - Total Sales: $12071.69
--------------------------------------------------
Top 10 - Store 428 - Total Sales: $11911.75
--------------------------------------------------
